# 如何流式运行 Runnables

:::info 前置条件

本指南假设您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [LangChain 表达式语言](/docs/concepts/lcel)
- [输出解析器](/docs/concepts/output_parsers)

:::

流式传输对于使基于 LLM 的应用程序对终端用户感觉响应迅速至关重要。

重要的 LangChain 原语（如 [聊天模型](/docs/concepts/chat_models)、[输出解析器](/docs/concepts/output_parsers)、[提示](/docs/concepts/prompt_templates)、[检索器](/docs/concepts/retrievers) 和 [代理](/docs/concepts/agents)）实现了 LangChain 的 [Runnable 接口](/docs/concepts/runnables)。

此接口提供了两种流式传输内容的通用方法：

1. 同步 `stream` 和异步 `astream`：一种流式传输的**默认实现**，用于从链中流式传输**最终输出**。
2. 异步 `astream_events` 和异步 `astream_log`：这些方法提供了一种流式传输链中**中间步骤**和**最终输出**的方式。

让我们来看看这两种方法，并尝试了解如何使用它们。

:::info
有关 LangChain 中流式传输技术的更高级别概述，请参阅[概念指南的这一部分](/docs/concepts/streaming)。
:::

## 使用 Stream

所有 `Runnable` 对象都实现了一个名为 `stream` 的同步方法和一个异步变体 `astream`。

这些方法旨在以块的形式流式传输最终输出，并在块可用时立即生成每个块。

只有当程序中的所有步骤都知道如何处理**输入流**时，流式传输才有可能；即一次处理一个输入块，并生成相应的输出块。

这种处理的复杂性可能会有所不同，从简单的任务（如发出 LLM 生成的标记）到更具挑战性的任务（如在整个 JSON 完成之前流式传输 JSON 结果的一部分）。

探索流式传输的最佳起点是 LLM 应用程序中最重要的组件——LLM 本身！

### LLM 和聊天模型

大型语言模型及其聊天变体是基于 LLM 的应用程序中的主要瓶颈。

大型语言模型可能需要**几秒钟**才能生成对查询的完整响应。这远远慢于**~200-300 毫秒**的阈值，在该阈值下应用程序对终端用户来说感觉响应迅速。

使应用程序感觉更响应的关键策略是显示中间进度；即**逐个标记**流式传输模型的输出。

我们将展示使用聊天模型进行流式传输的示例。从以下选项中选择一个：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="model"
/>


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

keys = [
    "ANTHROPIC_API_KEY",
    "OPENAI_API_KEY",
]

for key in keys:
    if key not in os.environ:
        os.environ[key] = getpass(f"Enter API Key for {key}=?")


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


让我们从同步 `stream` API 开始：

In [ ]:
chunks = []
for chunk in model.stream("天空是什么颜色？"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| sky| appears| blue| during| the| day|.|

或者，如果您在异步环境中工作，可以考虑使用异步 `astream` API：

In [ ]:
chunks = []
async for chunk in model.astream("天空是什么颜色？"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| sky| appears| blue| during| the| day|.|

让我们检查其中一个块

In [4]:
chunks[0]

AIMessageChunk(content='The', id='run-b36bea64-5511-4d7a-b6a3-a07b3db0c8e7')

我们得到了一个名为 `AIMessageChunk` 的东西。这个块表示 `AIMessage` 的一部分。

消息块是设计为可累加的——可以简单地将它们相加以获得到目前为止的响应状态！

In [5]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='The sky appears blue during', id='run-b36bea64-5511-4d7a-b6a3-a07b3db0c8e7')

### 链

几乎所有 LLM 应用程序都涉及比调用语言模型更多的步骤。

让我们使用 `LangChain 表达式语言` (`LCEL`) 构建一个简单的链，该链结合了提示、模型和解析器，并验证流式传输是否有效。

我们将使用 [`StrOutputParser`](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) 来解析模型的输出。这是一个简单的解析器，它从 `AIMessageChunk` 中提取 `content` 字段，给我们模型返回的 `token`。

:::tip
LCEL 是一种*声明式*方式，通过将不同的 LangChain 原语链接在一起来指定“程序”。使用 LCEL 创建的链受益于 `stream` 和 `astream` 的自动实现，允许流式传输最终输出。事实上，使用 LCEL 创建的链实现了整个标准 Runnable 接口。
:::

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("讲一个关于 {topic} 的笑话")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "鹦鹉"}):
    print(chunk, end="|", flush=True)

Here|'s| a| joke| about| a| par|rot|:|

A man| goes| to| a| pet| shop| to| buy| a| par|rot|.| The| shop| owner| shows| him| two| stunning| pa|rr|ots| with| beautiful| pl|um|age|.|

"|There|'s| a| talking| par|rot| an|d a| non|-|talking| par|rot|,"| the| owner| says|.| "|The| talking| par|rot| costs| $|100|,| an|d the| non|-|talking| par|rot| is| $|20|."|

The| man| says|,| "|I|'ll| take| the| non|-|talking| par|rot| at| $|20|."|

He| pays| an|d leaves| with| the| par|rot|.| As| he|'s| walking| down| the| street|,| the| par|rot| looks| up| at| him| an|d says|,| "|You| know|,| you| really| are| a| stupi|d man|!"|

The| man| is| stun|ne|d an|d looks| at| the| par|rot| in| dis|bel|ief|.| The| par|rot| continues|,| "|Yes|,| you| got| r|ippe|d off| big| time|!| I| can| talk| just| as| well| as| that| other| par|rot|,| an|d you| only| pai|d $|20| |for| me|!"|

请注意，即使我们在上面的链末尾使用了 `parser`，我们仍然获得了流式输出。`parser` 单独处理每个流式块。许多 [LCEL 原语](/docs/how_to#langchain-expression-language-lcel) 也支持这种变换式的透传流式传输，这在构建应用程序时非常方便。

可以[设计自定义函数以返回生成器](/docs/how_to/functions#streaming)，这些生成器能够处理流。

某些 runnables（如[提示模板](/docs/how_to#prompt-templates)和[聊天模型](/docs/how_to#chat-models)）无法处理单个块，而是聚合所有先前步骤。这些 runnables 可能会中断流式传输过程。

:::note
LangChain 表达式语言允许您将链的构建与其使用模式（例如，同步/异步、批处理/流式传输等）分开。如果这与您正在构建的内容无关，您还可以通过
分别调用每个组件上的 `invoke`、`batch` 或 `stream`，将结果分配给变量，然后根据需要在下游使用它们，依赖标准**命令式**编程方法。

:::

### 处理输入流

如果您想在生成时流式传输 JSON，该怎么办？

如果您依赖 `json.loads` 来解析部分 JSON，由于部分 JSON 不是有效的 JSON，解析将失败。

您可能会完全不知道该怎么做，并声称无法流式传输 JSON。

事实证明，有一种方法可以做到这一点——解析器需要处理**输入流**，并尝试将部分 JSON“自动完成”为有效状态。

让我们看看这样的解析器是如何工作的，以了解这意味着什么。

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)  # 由于旧版本 Langchain 中的一个错误，JsonOutputParser 未从某些模型流式传输结果
async for text in chain.astream(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 67}]}
{'countries': [{'name': 'France', 'population': 67413}]}
{'countries': [{'name': 'France', 'population': 67413000}]}
{'countries': [{'name': 'France', 'population': 67413000}, {}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351567}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 47351567}, {}]}
{'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain', 'population': 4735156

现在，让我们**打破**流式传输。我们将使用前面的示例，并在末尾附加一个提取函数，从最终的 JSON 中提取国家名称。

:::warning
链中的任何步骤如果操作**最终输入**而不是**输入流**，都可能通过 `stream` 或 `astream` 中断流式传输功能。
:::

:::tip
稍后，我们将讨论 `astream_events` API，它从中间步骤流式传输结果。即使链包含仅操作**最终输入**的步骤，此 API 也会从中间步骤流式传输结果。
:::

In [ ]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)


# 一个操作最终输入而不是输入流的函数
# 它打破了流式传输
def _extract_country_names(inputs):
    """一个不操作输入流并打破流式传输的函数。"""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

#### 生成器函数

让我们使用一个可以操作**输入流**的生成器函数来修复流式传输。

:::tip
生成器函数（使用 `yield` 的函数）允许编写操作**输入流**的代码
:::

In [ ]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """一个操作输入流的函数。"""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

:::note
由于上述代码依赖于 JSON 自动完成，您可能会看到国家名称的部分名称（例如 `Sp` 和 `Spain`），这不是提取结果所希望的！

我们专注于流式传输概念，而不一定是链的结果。
:::

### 非流式组件

某些内置组件（如检索器）不提供任何`流式传输`。如果我们尝试对它们进行`流式传输`会发生什么？🤨

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """仅根据以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison 在 kensho 工作过", "harrison 喜欢辛辣食物"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("harrison 在哪里工作过？")]
chunks

[[Document(page_content='harrison worked at kensho'),
  Document(page_content='harrison likes spicy food')]]

Stream 仅从该组件生成最终结果。

这没问题 🥹！并非所有组件都必须实现流式传输——在某些情况下，流式传输要么不必要，要么困难，要么没有意义。

:::tip
使用非流式组件构建的 LCEL 链，在很多情况下仍然能够流式传输，流式传输部分输出通常从链中的最后一个非流式步骤开始。
:::

In [11]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
for chunk in retrieval_chain.stream(
    "harrison 在哪里工作过？ " "写 3 个关于这个地方的虚构句子。"
):
    print(chunk, end="|", flush=True)

Base|d on| the| given| context|,| Harrison| worke|d at| K|ens|ho|.|

Here| are| |3| |made| up| sentences| about| this| place|:|

1|.| K|ens|ho| was| a| cutting|-|edge| technology| company| known| for| its| innovative| solutions| in| artificial| intelligence| an|d data| analytics|.|

2|.| The| modern| office| space| at| K|ens|ho| feature|d open| floor| plans|,| collaborative| work|sp|aces|,| an|d a| vib|rant| atmosphere| that| fos|tere|d creativity| an|d team|work|.|

3|.| With| its| prime| location| in| the| heart| of| the| city|,| K|ens|ho| attracte|d top| talent| from| aroun|d the| worl|d,| creating| a| diverse| an|d dynamic| work| environment|.|

现在我们已经了解了 `stream` 和 `astream` 的工作原理，让我们进入流式事件的世界。 🏞️

## 使用 Stream Events

事件流式传输是一个**测试版** API。根据反馈，此 API 可能会有所变化。

:::note

本指南演示了 `V2` API，需要 langchain-core >= 0.2。有关与旧版本 LangChain 兼容的 `V1` API，请参阅[此处](https://python.langchain.com/v0.1/docs/expression_language/streaming/#using-stream-events)。
:::

In [ ]:
import langchain_core

langchain_core.__version__

为了使 `astream_events` API 正常工作：

* 尽可能在代码中使用 `async`（例如，异步工具等）
* 如果定义自定义函数/运行对象，请传播回调
* 如果未使用 LCEL 的情况下使用运行对象，请确保在 LLM 上调用 `.astream()` 而不是 `.ainvoke` 以强制 LLM 流式传输标记。
* 如果有任何问题，请告诉我们！ :)

### 事件参考

以下是一个参考表，显示了各种 Runnable 对象可能发出的某些事件。


:::note
当流式传输实现正确时，运行对象的输入在输入流完全消耗之前将未知。这意味着 `inputs` 通常仅包含在 `end` 事件中，而不是 `start` 事件中。
:::

| 事件                | 名称             | 块                           | 输入                                         | 输出                                          |
|----------------------|------------------|---------------------------------|-----------------------------------------------|-------------------------------------------------|
| on_chat_model_start  | [模型名称]     |                                 | \{"messages": [[SystemMessage, HumanMessage]]\} |                                                 |
| on_chat_model_stream | [模型名称]     | AIMessageChunk(content="hello") |                                               |                                                 |
| on_chat_model_end    | [模型名称]     |                                 | \{"messages": [[SystemMessage, HumanMessage]]\} | AIMessageChunk(content="hello world")           |
| on_llm_start         | [模型名称]     |                                 | \{'input': 'hello'\}                            |                                                 |
| on_llm_stream        | [模型名称]     | 'Hello'                         |                                               |                                                 |
| on_llm_end           | [模型名称]     |                                 | 'Hello human!'                                |                                                 |
| on_chain_start       | format_docs      |                                 |                                               |                                                 |
| on_chain_stream      | format_docs      | "hello world!, goodbye world!"  |                                               |                                                 |
| on_chain_end         | format_docs      |                                 | [Document(...)]                               | "hello world!, goodbye world!"                  |
| on_tool_start        | some_tool        |                                 | \{"x": 1, "y": "2"\}                            |                                                 |
| on_tool_end          | some_tool        |                                 |                                               | \{"x": 1, "y": "2"\}                              |
| on_retriever_start   | [检索器名称] |                                 | \{"query": "hello"\}                            |                                                 |
| on_retriever_end     | [检索器名称] |                                 | \{"query": "hello"\}                            | [Document(...), ..]                             |
| on_prompt_start      | [模板名称]  |                                 | \{"question": "hello"\}                         |                                                 |
| on_prompt_end        | [模板名称]  |                                 | \{"question": "hello"\}                         | ChatPromptValue(messages: [SystemMessage, ...]) |

### 聊天模型

让我们首先看看聊天模型生成的事件。

In [ ]:
events = []
async for event in model.astream_events("你好"):
    events.append(event)

:::note

对于 `langchain-core<0.3.37`，显式设置 `version` 参数（例如，`model.astream_events("hello", version="v2")`）。

:::

让我们看看几个开始事件和几个结束事件。

In [14]:
events[:3]

[{'event': 'on_chat_model_start',
  'data': {'input': 'hello'},
  'name': 'ChatAnthropic',
  'tags': [],
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'name': 'ChatAnthropic',
  'tags': [],
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 8, 'output_tokens': 4, 'total_tokens': 12, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': 'b1

In [15]:
events[-2:]

[{'event': 'on_chat_model_stream',
  'run_id': 'b18d016d-8b9b-49e7-a555-44db498fcf66',
  'name': 'ChatAnthropic',
  'tags': [],
  'metadata': {'ls_provider': 'anthropic',
   'ls_model_name': 'claude-3-sonnet-20240229',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 1024},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 0, 'output_tokens': 12, 'total_tokens': 12, 'input_token_details': {}})},
  'parent_ids': []},
 {'event': 'on_chat_model_end',
  'data': {'output': AIMessageChunk(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-b18d016d-8b9b-49e7-a555-44db498fcf66', usage_metadata={'input_tokens': 8, 'output_tokens': 16, 'total_tokens': 24, 'input_token_details': {'cache_creation': 0, 'cache_read':

### 链

让我们重新审视解析流式 JSON 的示例链，以探索流式事件 API。

In [ ]:
chain = (
    model | JsonOutputParser()
)  # 由于旧版本 Langchain 中的一个错误，JsonOutputParser 未从某些模型流式传输结果

events = [
    event
    async for event in chain.astream_events(
        "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
        '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
        "每个国家应该有键 `name` 和 `population`",
    )
]

如果您检查前几个事件，您会注意到有**3**个不同的开始事件，而不是**2**个开始事件。

这三个开始事件对应于：

1. 链（模型 + 解析器）
2. 模型
3. 解析器

In [18]:
events[:3]

[{'event': 'on_chain_start',
  'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': '4765006b-16e2-4b1d-a523-edd9fd64cb92',
  'metadata': {}},
 {'event': 'on_chat_model_start',
  'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`')]]}},
  'name': 'ChatAnthropic',
  'tags': ['seq:step:1'],
  'run_id': '0320c234-7b52-4a14-ae4e-5f100949e589',
  'metadata': {}},
 {'event': 'on_chat_model_stream',
  'data': {'chunk': AIMessageChunk(content='{', id='run-0320c234-7b52-4a14-ae4e-5f100949e589')},
  'run_id': '0320c

如果您查看最后 3 个事件，您会看到什么？中间的事件呢？

让我们使用此 API 输出模型和解析器的流式事件。我们忽略了开始事件、结束事件和链的事件。

In [ ]:
num_events = 0

async for event in chain.astream_events(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"聊天模型块: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"解析器块: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # 截断输出
        print("...")
        break

Chat model chunk: ''
Chat model chunk: '{'
Parser chunk: {}
Chat model chunk: '\n  "countries'
Chat model chunk: '": [\n    '
Parser chunk: {'countries': []}
Chat model chunk: '{\n      "'
Parser chunk: {'countries': [{}]}
Chat model chunk: 'name": "France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n      "'
Chat model chunk: 'population": 67'
Parser chunk: {'countries': [{'name': 'France', 'population': 67}]}
Chat model chunk: '413'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413}]}
Chat model chunk: '000\n    },'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}]}
Chat model chunk: '\n    {'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {}]}
Chat model chunk: '\n      "name":'
...


由于模型和解析器都支持流式传输，我们实时看到来自两个组件的流式事件！很酷吧？🦜

### 过滤事件

由于此 API 生成了许多事件，因此能够过滤事件非常有用。

您可以按组件`名称`、组件`标签`或组件`类型`进行过滤。

#### 按名称

In [ ]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`",
    include_names=["my_parser"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # 截断输出
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'metadata': {}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['5a0bc625-09fd-4bdf-9932-54909a9a8c29']}
{'event': 'on_parser_stream', 'run_id': '37ee9e85-481c-415e-863b-c9e132d24948', 'name': 'my_parse

#### 按类型

In [ ]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    '以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。使用一个外部键为 "countries" 的字典，其中包含一个国家列表。每个国家应该有键 `name` 和 `population`',
    include_types=["chat_model"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # 截断输出
        print("...")
        break

{'event': 'on_chat_model_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'model', 'tags': ['seq:step:1'], 'run_id': '156c3e40-82fb-49ff-8e41-9e998061be8c', 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-sonnet-20240229', 'ls_model_type': 'chat', 'ls_temperature': 0.0, 'ls_max_tokens': 1024}, 'parent_ids': ['7b927055-bc1b-4b50-a34c-10d3cfcb3899']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-156c3e40-82fb-49ff-8e41-9e998061be8c', usage_metadata={'input_tokens': 56, 'output_tokens': 1, 'total_tokens': 57, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})}, 'run_id': '156c3e40-82fb-49ff-8e41-9e998061be8c', 'name': 'model', 'tags': ['seq:step:1'], 'metada

#### 按标签

:::caution

标签由给定运行对象的子组件继承。

如果您使用标签进行过滤，请确保这是您想要的。
:::

In [ ]:
chain = (model | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    '以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。使用一个外部键为 "countries" 的字典，其中包含一个国家列表。每个国家应该有键 `name` 和 `population`',
    include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # 截断输出
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '58d1302e-36ce-4df7-a3cb-47cb73d57e44', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatAnthropic', 'tags': ['seq:step:1', 'my_chain'], 'run_id': '8222e8a1-d978-4f30-87fc-b2dba838774b', 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-sonnet-20240229', 'ls_model_type': 'ch

### 非流式组件

还记得某些组件由于不操作**输入流**而无法很好地流式传输吗？

虽然使用 `astream` 时此类组件可能会中断最终输出的流式传输，但 `astream_events` 仍会从支持流式传输的中间步骤生成流式事件！

In [ ]:
# 不支持流式传输的函数。
# 它操作最终输入而不是
# 操作输入流。
def _extract_country_names(inputs):
    """一个不操作输入流并打破流式传输的函数。"""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = (
    model | JsonOutputParser() | _extract_country_names
)  # 此解析器目前仅适用于 OpenAI

正如预期的那样，`astream` API 无法正常工作，因为 `_extract_country_names` 不操作流。

In [ ]:
async for chunk in chain.astream(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`",
):
    print(chunk, flush=True)

['France', 'Spain', 'Japan']


现在，让我们确认使用 astream_events 我们仍然可以看到来自模型和解析器的流式输出。

In [ ]:
num_events = 0

async for event in chain.astream_events(
    "以 JSON 格式输出法国、西班牙和日本的国家及其人口的列表。 "
    '使用一个外部键为 "countries" 的字典，其中包含一个国家列表。 '
    "每个国家应该有键 `name` 和 `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"聊天模型块: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"解析器块: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # 截断输出
        print("...")
        break

Chat model chunk: ''
Chat model chunk: '{'
Parser chunk: {}
Chat model chunk: '\n  "countries'
Chat model chunk: '": [\n    '
Parser chunk: {'countries': []}
Chat model chunk: '{\n      "'
Parser chunk: {'countries': [{}]}
Chat model chunk: 'name": "France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n      "'
Chat model chunk: 'population": 67'
Parser chunk: {'countries': [{'name': 'France', 'population': 67}]}
Chat model chunk: '413'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413}]}
Chat model chunk: '000\n    },'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}]}
Chat model chunk: '\n    {'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {}]}
Chat model chunk: '\n      "name":'
Chat model chunk: ' "Spain",'
Parser chunk: {'countries': [{'name': 'France', 'population': 67413000}, {'name': 'Spain'}]}
Chat model chunk: '\n      "population":'
Chat model chunk: ' 47'
Parser chunk: {'countrie

### 传播回调

:::caution
如果您在工具中调用运行对象，则需要将回调传播到运行对象；否则，不会生成任何流式事件。
:::

:::note
使用 `RunnableLambdas` 或 `@chains` 装饰器时，回调会在后台自动传播。
:::

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]


reverse_word = RunnableLambda(reverse_word)


@tool
def bad_tool(word: str):
    """不正确传播回调的自定义工具。"""
    return reverse_word.invoke(word)


async for event in bad_tool.astream_events("你好"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': 'ea900472-a8f7-425d-b627-facdef936ee8', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '77b01284-0515-48f4-8d7c-eb27c1882f86', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '77b01284-0515-48f4-8d7c-eb27c1882f86', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'ea900472-a8f7-425d-b627-facdef936ee8', 'name': 'bad_tool', 'tags': [], 'metadata': {}}


以下是正确传播回调的重新实现。您会注意到，现在我们也从 `reverse_word` 运行对象中获取事件。

In [ ]:
@tool
def correct_tool(word: str, callbacks):
    """正确传播回调的工具。"""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("你好"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': 'd5ea83b9-9278-49cc-9f1d-aa302d671040', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '44dafbf4-2f87-412b-ae0e-9f71713810df', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '44dafbf4-2f87-412b-ae0e-9f71713810df', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'd5ea83b9-9278-49cc-9f1d-aa302d671040', 'name': 'correct_tool', 'tags': [], 'metadata': {}}


如果您从 Runnable Lambdas 或 `@chains` 中调用运行对象，则回调会自动为您传递。

In [28]:
from langchain_core.runnables import RunnableLambda


async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '5cf26fc8-840b-4642-98ed-623dda28707a', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '5cf26fc8-840b-4642-98ed-623dda28707a', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_chain_stream', 'data': {'chunk': '43214321'}, 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '03b0e6a1-3e60-42fc-8373-1e7829198d80', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}


使用 `@chain` 装饰器：

In [29]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'metadata': {}}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '64fc99f0-5d7d-442b-b4f5-4537129f67d1', 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '64fc99f0-5d7d-442b-b4f5-4537129f67d1', 'name': 'reverse_word', 'tags': [], 'metadata': {}}
{'event': 'on_chain_stream', 'data': {'chunk': '43214321'}, 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '1bfcaedc-f4aa-4d8e-beee-9bba6ef17008', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}}


## 下一步

现在您已经了解了一些使用 LangChain 流式传输最终输出和内部步骤的方法。

要了解更多信息，请查看本节中的其他操作指南或[Langchain 表达式语言的概念指南](/docs/concepts/lcel/)。